In [18]:
# Importing necessary libraries from PyTorch
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models, datasets
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import torch.nn.functional as F
import numpy as np

# Importing other necessary packages
import numpy as np
from PIL import Image

# Optional: for visualization and additional utilities
import matplotlib.pyplot as plt


# Load Pre-trained Model

In [19]:
from torchvision import models
import torch.nn as nn

# Load the pretrained MobileNetV3 model
model = models.mobilenet_v3_large(pretrained=True)

# Freeze all layers in the network
for param in model.parameters():
    param.requires_grad = False


In [20]:
# Print the model architecture
print(model)

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

# Step 1: Load the data

In [21]:
from dataset_prep import *

In [22]:
# PARAMS
BATCH_SIZE = 500
NUM_WORKERS = 8
PIN_MEMORY = False
SHUFFLE = False # preprocessed data is already shuffled
LEARNING_RATE = 1e-3
NUM_EPOCHS = 100
PATIENCE = 5
# NUM_BATCHES = 10 this one when want to break early, train just one batch etc.
LOAD_MODEL = False

In [23]:
# Base path
base_path = '/Users/ruimaciel/Desktop/Barcelona/DL_Image/Final_project/data_prepped'

# Define the datasets with correct paths
train_ds = Prepped_Painting_Dataset(data_dir=os.path.join(base_path, 'train_prep'))
valid_ds = Prepped_Painting_Dataset(data_dir=os.path.join(base_path, 'val_prep'))
test_ds = Prepped_Painting_Dataset(data_dir=os.path.join(base_path, 'test_prep'))


train_loader = DataLoader(train_ds,
                          batch_size=BATCH_SIZE,
                          num_workers=NUM_WORKERS,
                          shuffle=SHUFFLE,
                          persistent_workers=True)


valid_loader = DataLoader(valid_ds,
                          batch_size=BATCH_SIZE,
                          num_workers=NUM_WORKERS,
                          shuffle=SHUFFLE,
                          persistent_workers=True)


test_loader = DataLoader(test_ds,
                          batch_size=BATCH_SIZE,
                          num_workers=NUM_WORKERS,
                          shuffle=SHUFFLE,
                          persistent_workers=True)

# Step 2: Unfreeze the Last Layer

In [24]:
# Assuming you have 'num_classes' as the number of your target classes
num_classes = 26  # Change this to your number of classes

# Replace the classifier - note for MobileNetV3, the final classifier is named 'classifier'
model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)

# Step 3: Move the Model to a GPU (Optional)

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

cpu


# Step 4: Define Loss Function and Optimizer

In [28]:
from torch.optim import Adam

criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = Adam(model.classifier[3].parameters(), lr=0.001)  # Optimize only the classifier


# Step 5: Training the Model

In [ ]:
# model.train()  # Set the model to training mode
# num_epochs = 5  # Number of training epochs

# # Lists to track losses
# train_losses = []
# val_losses = []

# for epoch in range(num_epochs):
#     # Training phase
#     model.train()  # Ensure the model is in training mode
#     running_train_loss = 0.0
#     for inputs, labels in train_loader:
#         inputs, labels = inputs.to(device), labels.to(device)  # Move data to the appropriate device

#         # Convert labels to one-hot encoding
#         labels = torch.nn.functional.one_hot(labels.squeeze(), num_classes=26).float()

#         # Forward pass
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)

#         # Backward and optimize
#         optimizer.zero_grad()  # Clear gradients w.r.t. parameters
#         loss.backward()  # Backward pass
#         optimizer.step()  # Perform a single optimization step

#         running_train_loss += loss.item()

#     epoch_train_loss = running_train_loss / len(train_loader)
#     train_losses.append(epoch_train_loss)

#     print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_train_loss:.4f}')

#     # Validation phase
#     model.eval()  # Set the model to evaluation mode
#     running_val_loss = 0.0
#     with torch.no_grad():
#         for inputs, labels in valid_loader:
#             inputs, labels = inputs.to(device), labels.to(device)

#             # Convert labels to one-hot encoding
#             labels = torch.nn.functional.one_hot(labels.squeeze(), num_classes=26).float()

#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
#             running_val_loss += loss.item()

#     epoch_val_loss = running_val_loss / len(valid_loader)
#     val_losses.append(epoch_val_loss)

#     print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {epoch_val_loss:.4f}')

# # Plotting training and validation loss
# plt.figure(figsize=(10, 5))
# plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss')
# plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Training and Validation Loss')
# plt.legend()
# plt.show()


In [38]:
from tqdm import tqdm

# Assume you have DataLoader 'train_loader' and 'valid_loader'
num_epochs = 5  # Number of training epochs

# Lists to track losses
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    # Training phase
    model.train()  # Ensure the model is in training mode
    running_train_loss = 0.0
    loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Training")

    for inputs, labels in loop:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to the appropriate device

        # Ensure labels are 1D tensors of class indices
        if labels.ndimension() > 1:
            labels = labels.squeeze()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()  # Clear gradients w.r.t. parameters
        loss.backward()  # Backward pass
        optimizer.step()  # Perform a single optimization step

        running_train_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    epoch_train_loss = running_train_loss / len(train_loader)
    train_losses.append(epoch_train_loss)

    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_train_loss:.4f}')

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    running_val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Ensure labels are 1D tensors of class indices
            if labels.ndimension() > 1:
                labels = labels.squeeze()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()

    epoch_val_loss = running_val_loss / len(valid_loader)
    val_losses.append(epoch_val_loss)

    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {epoch_val_loss:.4f}')

# Plotting training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss')
plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()


Epoch [1/5] Training: 100%|██████████| 5/5 [01:10<00:00, 14.01s/it, loss=0.4] 


Epoch [1/5], Training Loss: 1.0122
Epoch [1/5], Validation Loss: 1.9656


Epoch [2/5] Training: 100%|██████████| 5/5 [01:15<00:00, 15.06s/it, loss=0.387]


Epoch [2/5], Training Loss: 0.9649
Epoch [2/5], Validation Loss: 1.9564


Epoch [3/5] Training:  20%|██        | 1/5 [00:29<01:56, 29.17s/it, loss=1.1]

# Step 6: Evaluate the Model

In [37]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report

# Inference with Softmax and Metrics Calculation on Test Dataset
model.eval()  # Set the model to evaluation mode
softmax = nn.Softmax(dim=1)
all_labels = []
all_predictions = []

with torch.no_grad():
    for inputs, labels in test_loader:  # Assume you have a DataLoader 'test_loader'
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        probabilities = softmax(outputs)  # Apply softmax to get probabilities
        _, predicted = torch.max(probabilities, 1)  # Get the index of the max probability
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_predictions) * 100
print(f'Overall Accuracy of the model on the test images: {accuracy:.2f}%')

# Calculate precision, recall, f1-score, and support for each class
precision, recall, f1, support = precision_recall_fscore_support(all_labels, all_predictions, average=None)

# Print precision, recall, f1-score, and accuracy for each class
for i in range(num_classes):
    class_precision = precision[i]
    class_recall = recall[i]
    class_f1 = f1[i]
    class_support = support[i]
    class_accuracy = 100 * (np.sum((np.array(all_labels) == i) & (np.array(all_predictions) == i))) / class_support
    print(f'Class {i}:')
    print(f'  Precision: {class_precision:.4f}')
    print(f'  Recall: {class_recall:.4f}')
    print(f'  F1 Score: {class_f1:.4f}')
    print(f'  Accuracy: {class_accuracy:.2f}%')

# Print overall precision, recall, f1-score
overall_precision, overall_recall, overall_f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='macro')
print(f'Overall Precision: {overall_precision:.4f}')
print(f'Overall Recall: {overall_recall:.4f}')
print(f'Overall F1 Score: {overall_f1:.4f}')

# Print classification report
print("\nClassification Report:")
print(classification_report(all_labels, all_predictions, target_names=[f'Class {i}' for i in range(num_classes)]))



Overall Accuracy of the model on the test images: 37.31%
Class 0:
  Precision: 0.4000
  Recall: 0.2000
  F1 Score: 0.2667
  Accuracy: 500.00%
Class 1:
  Precision: 0.1667
  Recall: 0.1000
  F1 Score: 0.1250
  Accuracy: 600.00%
Class 2:
  Precision: 0.2353
  Recall: 0.4000
  F1 Score: 0.2963
  Accuracy: 1700.00%
Class 3:
  Precision: 0.0000
  Recall: 0.0000
  F1 Score: 0.0000
  Accuracy: 700.00%
Class 4:
  Precision: 0.6250
  Recall: 0.5000
  F1 Score: 0.5556
  Accuracy: 800.00%
Class 5:
  Precision: 0.4000
  Recall: 0.4000
  F1 Score: 0.4000
  Accuracy: 1000.00%
Class 6:
  Precision: 0.1667
  Recall: 0.1000
  F1 Score: 0.1250
  Accuracy: 600.00%
Class 7:
  Precision: 0.2857
  Recall: 0.4000
  F1 Score: 0.3333
  Accuracy: 1400.00%
Class 8:
  Precision: 0.3333
  Recall: 0.1000
  F1 Score: 0.1538
  Accuracy: 300.00%
Class 9:
  Precision: 0.4000
  Recall: 0.2000
  F1 Score: 0.2667
  Accuracy: 500.00%
Class 10:
  Precision: 0.2308
  Recall: 0.3000
  F1 Score: 0.2609
  Accuracy: 1300.00%
Cla

# Unfreezing another layer

In [ ]:
# Load the pretrained MobileNetV3 model
model = models.mobilenet_v3_large(pretrained=True)

# Freeze all layers in the network
for param in model.parameters():
    param.requires_grad = False

# Assuming you have 'num_classes' as the number of your target classes
num_classes = 10  # Change this to your number of classes

# Replace the classifier - note for MobileNetV3, the final classifier is named 'classifier'
model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)

# Unfreeze the penultimate and the last layer
for param in model.classifier[0].parameters():
    param.requires_grad = True

for param in model.classifier[3].parameters():
    param.requires_grad = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)  # Only optimize parameters that require gradients

# Training loop
model.train()  # Set the model to training mode
num_epochs = 5  # Number of training epochs

for epoch in range(num_epochs):
    for inputs, labels in train_loader:  # Assume you have a DataLoader 'train_loader'
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to the appropriate device

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()  # Clear gradients w.r.t. parameters
        loss.backward()  # Backward pass
        optimizer.step()  # Perform a single optimization step

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Inference with Softmax and Metrics Calculation
model.eval()  # Set the model to evaluation mode
softmax = nn.Softmax(dim=1)
all_labels = []
all_predictions = []

with torch.no_grad():
    for inputs, labels in valid_loader:  # Assume you have a DataLoader 'valid_loader'
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        probabilities = softmax(outputs)  # Apply softmax to get probabilities
        _, predicted = torch.max(probabilities, 1)  # Get the index of the max probability
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_predictions) * 100
print(f'Overall Accuracy of the model on the test images: {accuracy:.2f}%')

# Calculate precision, recall, f1-score, and support for each class
precision, recall, f1, support = precision_recall_fscore_support(all_labels, all_predictions, average=None)

# Print precision, recall, f1-score, and accuracy for each class
for i in range(num_classes):
    class_accuracy = 100 * (np.array(all_labels) == np.array(all_predictions)).sum() / support[i]
    print(f'Class {i}:')
    print(f'  Precision: {precision[i]:.4f}')
    print(f'  Recall: {recall[i]:.4f}')
    print(f'  F1 Score: {f1[i]:.4f}')
    print(f'  Accuracy: {class_accuracy:.2f}%')

# Optionally, you can print the overall precision, recall, f1-score
overall_precision, overall_recall, overall_f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='macro')
print(f'Overall Precision: {overall_precision:.4f}')
print(f'Overall Recall: {overall_recall:.4f}')
print(f'Overall F1 Score: {overall_f1:.4f}')